## Token和Langchain总览
#### Token的概念
在开始Langchain之前，先补充一个重要的概念：Token。相信大家平时已经注意到了，不管是ChatGPT还是其他什么模型，只要是收费的服务，它的计费单位一般都是每1000token多少钱。而语言模型一般来说都有个上下文大小，比如ChatGPT是4096，GPT4默认是8K，最大可到32K。这里的单位也是token。比如使用ChatGPT时，你的问题和回答之和最多只能有4096个token。

在之前分享wolfram的《这就是ChatGPT》这本书的时候曾经说过NLP模型的基本原理都是根据语料库的概率一次预测一个词：
  
<center>
  <img src="./images/token.png">
</center>

其实这里的词严格来说就是token。ChatGPT的官方提供了[一个网站](https://platform.openai.com/tokenizer)可以可视化token

<center>
  <img src="./images/tokenizer.png">
</center>

我们可以通过下面的程序来证明，语言模型确实是一个token一个token输出的。这里使用了langchain的流式输出，每次for循环就表示LLM进行了一次输出，我们将每次的输出间隔符写成`|`，然后和上图的Tokenizer结果对比就会发现每次输出刚好是对应一个token。因此在transformer中，token是最小的输出单位。

In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model='qwen:14b')
for chunk in llm.stream("hello"):
    print(chunk, end='|', flush=True)

Hello| there|!| How| can| I| assist| you| today|?||

那么为什么语言模型有最大的token限制呢。这主要是因为几个原因：
1. 模型规模和token数量在正常情况下是线性关系的，处理token越多，计算开销越大
2. 输入和中间结果需要存放在内存中，token过大会导致占用更多内存
3. 输入太大会反而影响推理的效率和准确性

#### langchain
上次分享结尾已经提过langchain，应该都知道它是一个LLM应用开发框架。一般来说，我们把一个库能称为框架，肯定是因为它做了某种抽象，提出了某种开发的模型。拿大家比较熟悉的前端的例子，在前端早期，那时候基本都在使用jQuery。我们很少将jQuery称为一个框架，是因为它虽然封装了很多很有用的函数，但是它的思维方式是和用原生操作DOM的思路是一样的，没有进行任何抽象。而后来的vue，react等，都有自己的模型和抽象，因此我们都把这些称为框架。

同样的langchain虽然内置了很多工具方法，这些方法不用langchain你自己也可以去实现，但是它提出了一个自己的抽象，即所谓的chain，或者Runnable接口，这个是我们后面分享的一个重点，也是langchain的一个核心思想，在此之前我们先来总览下langchain的各个部分：

<center>
  <img src="./images/langchain.png">
</center>

上图是从官网截取的一个langchain的生态架构图，可以看出langchain的生态主要分为四块：

##### LangSmith
这是langchain提供的调试，监控，追踪，分析，评估LLM应用的一个开发平台，可以免费限量使用，没有开放源码，私有部署需要付费，普通用户可以免费限额使用。建议在自己学习langchain的时候都把这个对接上，可以让你观察到很多关于chain的中间状态，让你更清晰地知道你的整个链路的运行中间过程。

##### LangServe
这个是langchain提供的简化LLM部署的工具，只提供了python版本。个人感觉相比之下是没有其他组件重要，毕竟开发完成之后封装成Restful接口即使没有这个，使用python的一些web框架也可以很容易做到，当然它也提供了一些有意思的方便的功能。

##### Template
这个在langchain0.1.x时代是没有的，也是最近0.2.0才有的部分。它其实就是个脚手架模板的市场，上面有很多官方或者社区分享的可以快速搭建一个结构的脚手架，比如你可以选择一个openai+rag的Template，快速生成一个使用ChatGPT的RAG应用框架。

##### Langchain
这是Langchain的核心。从图上也可以看出，它也大概可以分为Langchain，Langchain_Community和Langchain_Core三个部分。官方维护了python和js两个版本。其中js版本可以用于浏览器和node环境，主要是提供给前端开发者使用。当然社区也有其他语言版本，java，golang，rust啥的，但是这些基本都是个人维护，没有官方的稳定，如果要使用建议慎重。
开始详细介绍Langchain前要吐槽一点，就是个人学习Langchain过程中觉得最难受的就是这几个包各自承载了什么功能很乱。虽然LLM应用开发本身确实是一个新的技术领域，所以langchain这种框架的api一直在与时俱进的调整是很正常的，但是在使用过程中你会发现某个模型在langchain下，某个模型在langchain_community下，甚至某些模型还有单独的包，就非常的混乱。加上python和js本来作为一个弱类型语言，提示也没有强类型语言那么准确，因此在学习过程中，多查官方文档是很有必要的。